In [1]:
#Configuración inicial del archivo

#Invocando librerías necesarias
import os
import numpy as np
import pandas as pd    #para trabajar con las matrices

#Para cambiar el directorio de trabajo:
#os.chdir('L:\Google Drive\Consultoría\Internexa')
os.chdir('L:\Google Drive\Consultoría\Internexa')
#os.chdir('C:/Users/CJ')


In [2]:
#Datos de entrada
#Ingreso de parámetros iniciales
ano_analisis = 2018
uso_max_transformador=100
costo_hora_desmontaje = 200000
costo_hora_montaje = 200000
costo_hora_transporte = 100000
tiempo_de_recuperacion = 12
velocidad = 30

In [3]:
##########################################
#  Definición de funciones #
##########################################

#Obtener los parámetros un nodo
def parametros_nodo(nodo):
    carga_max_nodo = float(nodos[nodos['id_nodo']==nodo]['carga_maxima_kVA'])
    ddames = float(nodos[nodos['id_nodo']==nodo]['demanda_mes_kVA'])
    tension = int(nodos[nodos['id_nodo']==nodo]['tension_nodo']) 
    costokwh = float(tarifadistribuidor[tarifadistribuidor['tension_nodo']==tension]['tarifa_kWh']) 
    return (carga_max_nodo,ddames,tension,costokwh)

def parametros_transformador(transf):
    capacidad_transf = float(inventario[inventario['id_transformador']==transf]['capacidad_kVA'])
    fc  = str(int(inventario[inventario['id_transformador']==transf]['fases'])) + '-' \
        + str(float(inventario[inventario['id_transformador']==transf]['capacidad_kVA']))
    ffc = str(int(inventario[inventario['id_transformador']==transf]['fabricante'])) + '-' + fc
    vida_util_t = int(inventario[inventario['id_transformador']==transf]['vida_util_teorica_anos'])
    costo_nuevo = int(caractecntransf[caractecntransf['fasesycapacidad']==fc]['costo_a_nuevo'])
    nodo_actual = int(inventario[inventario['id_transformador']==transf]['id_nodo'])
    return (capacidad_transf,fc,ffc,vida_util_t,costo_nuevo,nodo_actual)

#Costo de las pérdidas técnicas de un par
def costo_pertec_y_vutil(nodo,transf):
    
    ###
    
    ddames = float(nodos[nodos['id_nodo']==nodo]['demanda_mes_kVA'])
    
    carga_max_nodo = float(nodos[nodos['id_nodo']==nodo]['carga_maxima_kVA'])
    
    tension = int(nodos[nodos['id_nodo']==nodo]['tension_nodo']) 
    
    ###
    
    ## key -> fabricante + fases + capacidad_kVA  
    fc  = str(int(inventario[inventario['id_transformador']==transf]['fases'])) + '-' \
        + str(float(inventario[inventario['id_transformador']==transf]['capacidad_kVA']))
    ffc = str(int(inventario[inventario['id_transformador']==transf]['fabricante'])) + '-' + fc
    
    capacidad_transf = float(inventario[inventario['id_transformador']==transf]['capacidad_kVA'])
    usomax = int(inventario[inventario['id_transformador']==transf]['uso_max_transf'])
    
    
    costo_nuevo = int(caractecntransf[caractecntransf['fasesycapacidad']==fc]['costo_a_nuevo'])
    
    
    costokwh = float(tarifadistribuidor[tarifadistribuidor['tension_nodo']==tension]['tarifa_kWh']) 
    
    ###se
    
    pertecnicas = float(costos[costos['fabricanteyfasesycapacidadyuso']==ffcu]['perdidas_tecnicas'])
    #pervidautil = float(costos[costos['fabricanteyfasesycapacidadyuso']==ffcu]['perdida_vida_util'])
    pervidautil = float(costos[costos['fabricanteyfasesycapacidadyuso']==ffcu]['perdida_vida_util'])
    
    
    ### Cálculos intermedios
    
    anos_restantes = vida_util_t - anos_uso
    uso_transf= int(round(min(carga_max_nodo / capacidad_transf * 100,usomax),0))
    ffcu = str(ffc) + '-' + str(uso_transf)    
    
    #Pérdidas
    

    #Costo pérdidas técnicas
    kwhperdidos = ddames * pertecnicas    
    costo_perdidas_tecnicas = float(kwhperdidos * costokwh)

    #Vida útil
    vida_util_real = vida_util_t  * (1-pervidautil)
    costo_vida_util = ((costo_nuevo / vida_util_t) * pervidautil *  anos_restantes) / 12
    
    #total
    ctotal = costo_perdidas_tecnicas + costo_vida_util
    
    return (ctotal,costo_perdidas_tecnicas,costo_vida_util)
    
    
    
def distancia_nodos(nodo1,nodo2):
    distancia =distancias[(distancias['Nodo1']==nodo1) & (distancias['Nodo2']==nodo2)]['km']
    if distancia.empty:
        distancia =distancias[(distancias['Nodo1']==nodo2) & (distancias['Nodo2']==nodo1)]['km']
    dist = float(distancia)  
    return dist  
    
def tiempo_mont(transf):
    fc  = str(int(inventario[inventario['id_transformador']==transf]['fases'])) + '-' \
        + str(float(inventario[inventario['id_transformador']==transf]['capacidad_kVA']))
    tiempo_mon = float(caractecntransf[caractecntransf['fasesycapacidad']==fc]['tiempo_montaje_h'])
    return tiempo_mon

def tiempo_desm(transf):
    fc  = str(int(inventario[inventario['id_transformador']==transf]['fases'])) + '-' \
        + str(float(inventario[inventario['id_transformador']==transf]['capacidad_kVA']))
    tiempo_desmon = float(caractecntransf[caractecntransf['fasesycapacidad']==fc]['tiempo_desmontaje_h'])
    return tiempo_desmon

def tiempo_residual(transf):
    vida_util_t = int(inventario[inventario['id_transformador']==transf]['vida_util_teorica_anos'])
    anos_uso = float(inventario[inventario['id_transformador']==transf]['anos_de_uso'])
    anos_restantes = vida_util_t - anos_uso
    return anos_restantes

def costo_permutacion(distancia,velocidad,tiempo_mon1,tiempo_mon2,tiempo_desmon1,tiempo_desmon2,costoxhoratte,costoxhoramon,costoxhoradesm):
    costo_transporte = distancia / velocidad  * costoxhoratte * 2
    costo_montaje = (tiempo_mon1 + tiempo_mon2) * costoxhoramon
    costo_desmonte = (tiempo_desmon1 + tiempo_desmon2) * costoxhoradesm
    costototal = costo_transporte + costo_montaje + costo_desmonte
    return costototal

In [4]:
#################Lectura de los datos de entrada######################
xls = pd.ExcelFile("datos_entrada.xlsx")   
nodos = xls.parse("nodos",usecols = 'A:H')
inventario_transf = xls.parse("inventario_transformadores", usecols = "A:G")
caractecntransf = xls.parse("carac_tecn_transf",usecols = "A:E")
costos = xls.parse("costos",usecols = "A:G")
tarifadistribuidor = xls.parse("tarifa_distribuidor",usecols = "A:B")
tarifascreg = xls.parse("tarifas_creg",usecols = "A:C")
distancias = xls.parse("distancias",usecols = "A:C")

#Concatenar
costos['fabricanteyfasesycapacidadyuso']=costos['fabricante'].map(str) + "-" + costos['fases'].map(str) + "-" \
                                        + costos['capacidad_kVA'].map(str) + "-" + costos['uso_transf'].map(str)
inventario_transf['uso_max_transf'] = uso_max_transformador
caractecntransf['fasesycapacidad'] = caractecntransf['fases'].map(str) + "-" + caractecntransf['capacidad_kVA'].map(str)

inventario = inventario_transf 

#print (inventario)

In [5]:
###################################################################
###Módulo de evaluación de costos de la solución actual ###########
###################################################################

nnodos = nodos.shape[0]
ntransformadores = inventario.shape[0]
inventario = inventario_transf.copy()
solucion_original = pd.DataFrame(columns=['nodo','transformador','costo','permutado','costo_perm'])

for i in range (0,nnodos):
    nodo_ev = int(nodos.iloc[i,0])
    transf_ev = int(inventario[inventario['id_nodo']==nodo_ev]['id_transformador'])
        
    #calcular los costos del par (nodo,transformador)
    ctotal,cper,cvutil = costo_pertec_y_vutil(nodo_ev,transf_ev)
    #almacenar el costo para ese arreglo en particular
    solucion_original.loc[i]= [nodo_ev,transf_ev,ctotal,0,0]

costo_de_la_solucion_original = solucion_original['costo'].sum()
#print(costo_de_la_solucion_original)
#print(solucion_original)

In [19]:
###################################################################
###Módulo de la iteración de nodos ###########
###################################################################

inventario = inventario_transf.copy()
solucion=solucion_original.copy()
solperm=solucion_original.copy()
permutaciones = 0
costo_operacion_sol=costo_de_la_solucion_original
info_permutaciones_per = pd.DataFrame(columns=['iteracion','nodo','trans_orig','transf_fin','costo_perm','costo_oper_sol','ctotal'])
info_permutaciones_iter = pd.DataFrame(columns=['iteracion','nodo','trans_orig','transf_fin','costo_perm','costo_oper_sol','ctotal'])
iteracion = 0

for i in range (0,nnodos):
#for i in range (0,1):
    #Ordenar la solución
    solucion = solucion.sort_values(['costo'],ascending=[False])
    
    #Elegir el nodos más costoso aún no evaluado para permutación
    nodo_seleccionado = 0
    for k in range (0,nnodos):
        if (solucion.iloc[k]['permutado'] == 0):
            nodo_seleccionado = 1
            #Información del nodo en evaluación
            nodo1 = int(solucion.iloc[k,0])
            carga_max_nodo1,ddames1,tension1,costokwh1 = parametros_nodo(nodo1)
        if nodo_seleccionado == 1: break
            
    #Buscar transformadores potenciales para ese nodo (para hacer permutación)
    #inventario=pd.merge(inventario, nodos[['carga_maxima_kVA','id_nodo']], on='id_nodo', how='left')
    tranf_potenciales = inventario.loc[inventario['capacidad_kVA']>= carga_max_nodo1]
    ntranfpot= tranf_potenciales.shape[0]
  #  print (tranf_potenciales)
    
    for j in range (0,ntranfpot):
#    for j in range (0,2):
        iteracion = iteracion + 1
        
        #Actualizar el nuevo transformador que hay en el nodo de interés (por si se hizo un reemplazo en algún j anterior)
        transf1 = int(solucion.iloc[k,1])
        capacidad_transf1,fc1,ffc1,vida_util_t1,costo_nuevo1,nodo1 = parametros_transformador(transf1)
        costo1 = int(solucion.iloc[k,2]) 
        
        #identificar el transformador potencial para reemplazar al original
        transf2 = int(tranf_potenciales.iloc[j]['id_transformador'])
        capacidad_transf2,fc2,ffc2,vida_util_t2,costo_nuevo2,nodo2 = parametros_transformador(transf2)   
        
        #La carga del nodo "bodega" es cero, para permitir que haga intercambio
        carga_max_nodo2 = 0
        if nodo2 !=0: carga_max_nodo2,ddames2,tension2,costokwh2 = parametros_nodo(nodo2)
        
        #Si se cumplen las restricciones de carga entre el transformador 1 y el nodo 2 
        if capacidad_transf1 > carga_max_nodo2:
            
            #Costos de operación del nodo 1 con el transformador 2, y reemplazo
            ctotal1,cper1,cvutil1 = costo_pertec_y_vutil(nodo1,transf2)
            solperm.at[solperm['nodo']==nodo1,'transformador']=transf2
            solperm.at[solperm['nodo']==nodo1,'costo']=ctotal1

            #Costos de operación del nodo 2 con el transformador 1, y reemplazo
            #Si el intercambio no es con la bodega, entonces hay que indicar el reemplazo en el otro nodo
            if nodo2 != 0:
                ctotal2,cper2,cvutil2 = costo_pertec_y_vutil(nodo2,transf1)
                solperm.at[solperm['nodo']==nodo2,'transformador']=transf1
                solperm.at[solperm['nodo']==nodo2,'costo']=ctotal2
            else:
                ctotal2=0
            
            #Calcular el costo de esta permutación agregada
            #Detectar los movimientos necesarios para llegar a esta solución
            solucion = solucion.sort_values(['nodo'],ascending=[True])
            solperm = solperm.sort_values(['nodo'],ascending=[True])
            
            for k in range (0,nnodos):
                transfantes = int(solucion_original.iloc[k]['transformador'])
                transfdesp = int(solperm.iloc[k]['transformador'])
                if transfantes != transfdesp:
                    tdesmont = tiempo_desm(transfdesp)
                    tmont = tiempo_mont(transfdesp)
                    anos_que_le_quedan = tiempo_residual(transfdesp)
                        
                    #Verificar si el nodo original de este transformador era bodega
                    if solucion_original[solucion_original['transformador']==transfdesp].empty:
                        nodoantes = 0
                    else:
                        nodoantes = int(solucion_original[solucion_original['transformador']==transfdesp]['nodo'])
                    nododesp = int(solperm.iloc[k]['nodo'])
                    distancia = distancia_nodos(nodoantes,nododesp)
                        
                    #Calcular el costo de esta permutación
                    costo_esta_permutacion =  tdesmont * costo_hora_desmontaje \
                                            + distancia / velocidad * costo_hora_transporte \
                                            + tmont * costo_hora_montaje
                    costo_esta_permutacion = costo_esta_permutacion / (anos_que_le_quedan * 12)
                        
                    #Guardar la permutación y su costo
                    solperm.at[k,'permutado']=1
                    solperm.at[k,'costo_perm']=costo_esta_permutacion                        
                        
            #Calcular el costo de todas las permutaciones realizadas para esta solución 
            costo_permutacion = solperm['costo_perm'].sum()
            
            #Cálculo de costos de esta nueva solución
            costo_operacion_perm = solperm['costo'].sum()
            
            #Verificar si conserva esta permutación
            if costo_operacion_perm + costo_permutacion < costo_operacion_sol :
                permutaciones = permutaciones + 1

                solucion=solperm.copy()
                costo_operacion_sol = costo_operacion_perm
                costo_perm_sol = costo_permutacion
                costo_total = costo_operacion_sol + costo_perm_sol 
                #afectar el nodo donde están ubicados los transformadores
                inventario.at[inventario['id_transformador']==transf1,'id_nodo']=nodo2
                inventario.at[inventario['id_transformador']==transf2,'id_nodo']=nodo1
                   
                #Guardar la información de esta pemutación realizada
                info_permutaciones_per.loc[permutaciones-1]= [iteracion,nodo1,transf1,transf2,costo_perm_sol,costo_operacion_sol,costo_total]
            else:
                #Deshacer la permutación
                solperm = solucion.copy()
            
            #Guardar la información de esta pemutación, realizada o no
            info_permutaciones_iter.loc[iteracion-1]= [iteracion,nodo1,transf1,transf2,costo_perm_sol,costo_operacion_sol,costo_total]
                
#Ordenar
solucion = solucion.sort_values(['nodo'],ascending=[True])
#Calcular cuántos y de qué capacidades estaban en bodega

#Impresiones
print(permutaciones)
print(costo_de_la_solucion_original,costo_operacion_sol,costo_perm_sol)
print(solucion_original)
print(solucion)

#exportar
solucion_original.to_csv('solucion_original.csv')
solucion.to_csv('solucion_final.csv')
info_permutaciones_per.to_csv('info_permutaciones_per.csv')
info_permutaciones_iter.to_csv('info_permutaciones_iter.csv')

#f = open('solucion.csv','w')
#f.write("display,variable x")
#f.close()

#print (inventario)

16
9952641.476695621 7797777.154606892 76629.38982549401
    nodo  transformador          costo  permutado  costo_perm
0    1.0           20.0  515145.248155        0.0         0.0
1    2.0           19.0  716497.507412        0.0         0.0
2    3.0           18.0  497251.380628        0.0         0.0
3    4.0           17.0  526138.195311        0.0         0.0
4    5.0           16.0  557567.472201        0.0         0.0
5    6.0           15.0  852879.688726        0.0         0.0
6    7.0           14.0  829501.949127        0.0         0.0
7    8.0           13.0  939938.359486        0.0         0.0
8    9.0           12.0  223212.014855        0.0         0.0
9   10.0           11.0  261987.660230        0.0         0.0
10  11.0           10.0  311782.430477        0.0         0.0
11  12.0            9.0  445162.121015        0.0         0.0
12  13.0            8.0  529639.095886        0.0         0.0
13  14.0            7.0  173317.230194        0.0         0.0
14  15.0     

In [20]:
bodega_antes = inventario_transf.loc[inventario_transf['id_nodo']==0]
cantidad_bodega_antes = bodega_antes.shape[0]
capacidad_bodega_antes = bodega_antes['capacidad_kVA'].sum()

bodega_desp = inventario.loc[inventario['id_nodo']==0]
cantidad_bodega_desp = bodega_desp.shape[0]
capacidad_bodega_desp = bodega_desp['capacidad_kVA'].sum()

#exportar
bodega_antes.to_csv('bodega_original.csv')
bodega_desp.to_csv('bodega_final.csv')

print(cantidad_bodega_antes,capacidad_bodega_antes,cantidad_bodega_desp,capacidad_bodega_desp)

10 480.0 10 1135.0


In [21]:
#seleccionar los transformadores de interés
carga = 100

a=inventario.loc[inventario['capacidad_kVA']>= carga]
print(a)

    id_transformador  fabricante  fases  capacidad_kVA  anos_de_uso  id_nodo  \
0                  1           1      3          150.0           11        8   
2                  3           1      3          250.0           20        9   
3                  4           1      3          150.0            4       16   
6                  7           1      3          112.5           17       14   
7                  8           1      3          150.0           15       13   
8                  9           1      3          250.0            4       12   
12                13           1      3          250.0            7       17   
13                14           1      3          250.0            4        0   
14                15           1      3          250.0            4        0   
15                16           1      3          250.0           18        5   
16                17           1      3          300.0           21        4   
17                18           1      3 

In [59]:
inventario

,id_transformador,fabricante,fases,capacidad_kVA,anos_de_uso,id_nodo,vida_util_teorica_anos,uso_max_transf,permutacion
0,1,1,3,150.0,11,8,25,100,0
1,2,1,3,30.0,3,19,25,100,0
2,3,1,3,250.0,20,9,25,100,0
3,4,1,3,150.0,4,16,25,100,0
4,5,1,3,75.0,1,6,25,100,0
5,6,1,3,75.0,7,15,25,100,0
6,7,1,3,112.5,17,14,25,100,0
7,8,1,3,150.0,15,13,25,100,0
8,9,1,3,250.0,4,12,25,100,0
9,10,1,3,75.0,6,11,25,100,0


In [22]:
bodega_desp

,id_transformador,fabricante,fases,capacidad_kVA,anos_de_uso,id_nodo,vida_util_teorica_anos,uso_max_transf
13,14,1,3,250.0,4,0,25,100
14,15,1,3,250.0,4,0,25,100
18,19,1,3,300.0,13,0,25,100
20,21,1,3,20.0,20,0,25,100
21,22,1,3,150.0,5,0,25,100
22,23,1,3,30.0,21,0,25,100
23,24,1,3,15.0,11,0,25,100
24,25,1,3,30.0,1,0,25,100
25,26,1,3,15.0,17,0,25,100
28,29,1,3,75.0,11,0,25,100
